## Setup

In [ ]:
# Necessary imports

from datetime import datetime, timedelta
import os
import json
import glob
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import pandas as pd
from scipy.stats import iqr
import shutil
import zipfile
from scipy.interpolate import interp1d
from scipy.ndimage import uniform_filter1d
from pathlib import Path

%matplotlib inline

In [ ]:
start_time = datetime.now()

In [ ]:
# Download data from OpenNeuro ⏳

!openneuro-py download --dataset ds005090 --target-dir data-phantom/

In [ ]:
# Define useful variables
path_data = os.path.join(os.getcwd(), "data-phantom/")
print(f"path_data: {path_data}")
path_labels = os.path.join(path_data, "derivatives", "labels")
path_qc = os.path.join(path_data, "qc")
subjects = [os.path.basename(subject_path) for subject_path in sorted(glob.glob(os.path.join(path_data, "sub-*")))]
print(f"subjects: {subjects}")

# Create output folder
path_results = os.path.join(path_data, "derivatives", "results")
os.makedirs(path_results, exist_ok=True)

## Convert TFL and DREAM flip angle maps to B1+ in units of nT/V 

In [ ]:
# load DREAM FA maps acquired with different reference voltages
# threshold FA maps to 20deg < FA < 50deg
# combine FA maps by averaging non-zero estimates of FA in each pixel

GAMMA = 2.675e8;  # [rad / (s T)]
voltages = ["1.5", "0.66"]

for subject in subjects:
    
    b1_maps = []
    os.chdir(os.path.join(path_data, subject, "fmap"))

    if subject=='sub-MSSM1':
        ref_voltage=450
    elif subject=='sub-MSSM2':
        ref_voltage=350
    elif subject=='sub-MSSM3':
        ref_voltage=450
    else:     
        # Fetch the reference voltage from the JSON sidecar 
        with open(f"{subject}_acq-famp_TB1DREAM.json", "r") as f:
            metadata = json.load(f)
            ref_voltage = metadata.get("TxRefAmp", "N/A")
            if (ref_voltage == "N/A"):
                ref_token = "N/A"
                for token in metadata.get("SeriesDescription", "N/A").split("_"):
                    if token.startswith("RefV"): ref_token = token
                ref_voltage = float(ref_token[4:-1])
    
    # Open refV flip angle map with nibabel
    nii = nib.load(f"{subject}_acq-famp_TB1DREAM.nii.gz")
    meas_fa = nii.get_fdata()
    #thresholding
    meas_fa[meas_fa < 200] = np.nan
    meas_fa[meas_fa > 500] = np.nan

    # Fetch the flip angle from the JSON sidecar 
    with open(f"{subject}_acq-famp_TB1DREAM.json", "r") as f:
        metadata = json.load(f)
        requested_fa = metadata.get("FlipAngle", "N/A")
        #convert measured FA to percent of requested FA (note that measured FA map is in degrees * 10)
        meas_fa = (meas_fa/10) / requested_fa

    # Account for the power loss between the coil and the socket. That number was given by Siemens.
    voltage_at_socket = ref_voltage * 10 ** -0.095
    # Compute B1 map in [T/V]
    b1_map = meas_fa * (np.pi / (GAMMA * 1e-3 * voltage_at_socket))
    # Convert to [nT/V]
    b1_map = b1_map * 1e9
    
    b1_maps.append(b1_map)

    for voltage in voltages:
        
        #check if map exists
        my_file = Path(f"{subject}_acq-famp-{voltage}_TB1DREAM.nii.gz")
        
        if my_file.is_file():
            
            if subject=='sub-MSSM2' and voltage=="1.5":
                ref_voltage=450
            elif subject=='sub-MSSM2' and voltage=="0.66":
                ref_voltage=234
            elif subject=='sub-MSSM3' and voltage=="0.66":
                ref_voltage=328
            else:            
                # Fetch the reference voltage from the JSON sidecar 
                with open(f"{subject}_acq-famp-{voltage}_TB1DREAM.json", "r") as f:
                    metadata = json.load(f)
                    ref_voltage = metadata.get("TxRefAmp", "N/A")
                    if (ref_voltage == "N/A"):
                        ref_token = "N/A"
                        for token in metadata.get("SeriesDescription", "N/A").split("_"):
                            if token.startswith("RefV"): ref_token = token
                        ref_voltage = float(ref_token[4:-1])
                
            # Open flip angle map with nibabel
            nii = nib.load(f"{subject}_acq-famp-{voltage}_TB1DREAM.nii.gz")
            meas_fa = nii.get_fdata()
            #thresholding
            meas_fa[meas_fa < 200] = np.nan
            meas_fa[meas_fa > 500] = np.nan
        
            # Fetch the flip angle from the JSON sidecar 
            with open(f"{subject}_acq-famp-{voltage}_TB1DREAM.json", "r") as f:
                metadata = json.load(f)
                requested_fa = metadata.get("FlipAngle", "N/A")
                #convert measured FA to percent of requested FA (note that measured FA map is in degrees * 10)
                meas_fa = (meas_fa/10) / requested_fa
        else:
            meas_fa = np.full((nii.header).get_data_shape(),np.nan)

        # Account for the power loss between the coil and the socket. That number was given by Siemens.
        voltage_at_socket = ref_voltage * 10 ** -0.095
        # Compute B1 map in [T/V]
        # Siemens maps are in units of flip angle * 10 (in degrees)
        b1_map = meas_fa * (np.pi / (GAMMA * 1e-3 * voltage_at_socket))
        # Convert to [nT/V]
        b1_map = b1_map * 1e9
        
        b1_maps.append(b1_map)
 
    # compute mean of non-zero values
    avgB1=np.nanmean(b1_maps,axis=0)
    
    # Save as NIfTI file
    nii_avgB1 = nib.Nifti1Image(avgB1, nii.affine, nii.header)
    nib.save(nii_avgB1, f"{subject}_DREAMTB1avgB1map.nii.gz")
    

In [ ]:
# Convert the TFL flip angle maps to B1+ efficiency maps [nT/V] (inspired by code from Kyle Gilbert)
# The approach consists in calculating the B1+ efficiency using a 1ms, pi-pulse at the acquisition voltage,
# then scale the efficiency by the ratio of the measured flip angle to the requested flip angle in the pulse sequence.

GAMMA = 2.675e8;  # [rad / (s T)]

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))

    if subject=='sub-MSSM1':
        ref_voltage=450
    elif subject=='sub-MSSM2':
        ref_voltage=350
    elif subject=='sub-MSSM3':
        ref_voltage=450
    else:     
        # Fetch the reference voltage from the JSON sidecar 
        with open(f"{subject}_acq-famp_TB1TFL.json", "r") as f:
            metadata = json.load(f)
            ref_voltage = metadata.get("TxRefAmp", "N/A")
            if (ref_voltage == "N/A"):
                ref_token = "N/A"
                for token in metadata.get("SeriesDescription", "N/A").split("_"):
                    if token.startswith("RefV"): ref_token = token
                ref_voltage = float(ref_token[4:-1])
        
    print(f"ref_voltage [V]: {ref_voltage} ({subject}_acq-famp_TB1TFL)")
                
    # Fetch the flip angle from the JSON sidecar 
    with open(f"{subject}_acq-famp_TB1TFL.json", "r") as f:
        metadata = json.load(f)
        requested_fa = metadata.get("FlipAngle", "N/A")
        print(f"flip angle [degrees]: {requested_fa} ({subject}_acq-famp_TB1TFL)")

    # Open flip angle map with nibabel
    nii = nib.load(f"{subject}_acq-famp_TB1TFL.nii.gz")
    meas_fa = nii.get_fdata()

    # Account for the power loss between the coil and the socket. That number was given by Siemens.
    voltage_at_socket = ref_voltage * 10 ** -0.095

    # Compute B1 map in [T/V]
    # Siemens maps are in units of flip angle * 10 (in degrees)
    b1_map = ((meas_fa / 10) / requested_fa) * (np.pi / (GAMMA * 1e-3 * voltage_at_socket))

    # Convert to [nT/V]
    b1_map = b1_map * 1e9

    # Save B1 map in [T/V] as NIfTI file
    nii_b1 = nib.Nifti1Image(b1_map, nii.affine, nii.header)
    nib.save(nii_b1, f"{subject}_TFLTB1map.nii.gz")


## Generate approximate mask of spinal cord

The purpose of this section is to generate an approximate mask of the spinal cord that will be used to quantify B1+ values inside the phantom. To do this, we identified a representative human subject, overlaid it on the anthropomorphic phantom, created a few pointwise labels along the spinal cord, interpolated those points using bsplines, and then created a cylindrical mask of 10mm diameter along the generated centerline.

![image](https://github.com/spinal-cord-7t/coil-qc-code/assets/2482071/185d823a-0c67-4c99-98e1-6cb52626c492)

In [ ]:
os.chdir(os.path.join(path_data, "sub-CRMBM", "fmap"))

# Create labels on TFL B1+ map along an approximate spinal cord
! sct_label_utils -i "sub-CRMBM_acq-anat_TB1TFL.nii.gz" -create 49,108,23,1:49,99,23,1:49,88,23,1:50,76,23,1:55,63,23,1:62,51,23,1:68,35,23,1:72,26,23,1 -o "label_cord.nii.gz"
# Create spinal cord centerline
! sct_get_centerline -i "label_cord.nii.gz" -method fitseg -o "centerline_spinalcord.nii.gz"
# Create mask of 10mm diameter around centerline
! sct_create_mask -i "sub-CRMBM_acq-anat_TB1TFL.nii.gz" -p centerline,"centerline_spinalcord.nii.gz" -size 10mm -f cylinder -o "mask_spinalcord_TFL.nii.gz"

## Co-register fmap data

The purpose of this section if to co-register phantom data across sites, to produce figures showing consistent placement of the phantom across sites, and to be able to use a single mask of the approximate spinal cord for quantitative analysis.

The co-registration uses programatically-created labels that correspond to the posterior tip of the phantom, at the inflexion point:

![image](https://github.com/spinal-cord-7t/coil-qc-code/assets/2482071/aa28085e-3e7d-4d16-9972-7559ea1027a9)

In [ ]:
# Create dictionary of labels across sites
TFL_labels_phantom = {
    'sub-CRMBM': '68,88,23',
    'sub-MGH': '61,98,26',
    'sub-MNI': '69,87,26',
    'sub-MPI': '67,95,27',
    'sub-MSSM': '73,65,28',
    'sub-NTNU': '64,96,29',
    'sub-UCL': '68,97,28',
}

DREAM_labels_phantom = {
    'sub-CRMBM': '63,55,5',
    'sub-MGH': '56,65,7',
    'sub-MNI': '56,54,5',
    'sub-MPI': '61,62,5',
    'sub-MSSM': '66,41,7',
    'sub-NTNU': '59,63,5',
    'sub-UCL': '63,61,5',
}

SNR_labels_phantom = {
    'sub-CRMBM': '291,282,6',
    'sub-MGH': '281,298,8',
    'sub-MNI': '279,279,6',
    'sub-MPI': '291,293,6',
    'sub-MSSM': '299,258,8',
    'sub-NTNU': '285,294,5',
    'sub-UCL': '294,291,5',
}

labels_phantom = [TFL_labels_phantom, DREAM_labels_phantom, SNR_labels_phantom]

subject_ref = 'sub-CRMBM'

file_names = ["TFLTB1map", "DREAMTB1avgB1map","acq-coilQaSagLarge_SNR_T0000"]

# Generate labels for the reference scan
os.chdir(os.path.join(path_data, subject_ref, "fmap"))
for file_name, label_phantom in zip(file_names,labels_phantom):
    # Create label
    !sct_label_utils -i {subject_ref}_{file_name}.nii.gz -create {label_phantom[subject_ref]},1 -o {subject_ref}_{file_name}_label.nii.gz
    
    # register TFL B1+ spinal cord mask to DREAM and SNR maps using those labels
    if file_name != "TFLTB1map":
        if file_name == "DREAMTB1avgB1map":
            !sct_register_multimodal -i mask_spinalcord_TFL.nii.gz -ilabel {subject_ref}_TFLTB1map_label.nii.gz -d {subject_ref}_{file_name}.nii.gz -dlabel {subject_ref}_{file_name}_label.nii.gz -param step=1,type=label,dof=Tx_Ty_Tz -o mask_spinalcord_DREAM.nii.gz
        else:
            !sct_register_multimodal -i mask_spinalcord_TFL.nii.gz -ilabel {subject_ref}_TFLTB1map_label.nii.gz -d {subject_ref}_{file_name}.nii.gz -dlabel {subject_ref}_{file_name}_label.nii.gz -param step=1,type=label,dof=Tx_Ty_Tz -o mask_spinalcord_SNR.nii.gz

# Generate labels and register to the reference scan
for subject in [other_subjects for other_subjects in subjects if other_subjects != subject_ref]:
    print(f"👉 PROCESSING: {subject}")
    for file_name, label_phantom in zip(file_names, labels_phantom):
        os.chdir(os.path.join(path_data, subject, "fmap"))

        if file_name=="acq-coilQaSagLarge_SNR_T0000":
            # Splitting SNR map 
            !sct_image -i {subject}_acq-coilQaSagLarge_SNR.nii.gz -split t -o {subject}_acq-coilQaSagLarge_SNR.nii.gz
                    
        # Create label
        !sct_label_utils -i {subject}_{file_name}.nii.gz -create {label_phantom[subject]},1 -o {subject}_{file_name}_label.nii.gz
        # Perform registration
        !sct_register_multimodal -i {subject}_{file_name}.nii.gz -ilabel {subject}_{file_name}_label.nii.gz -d ../../{subject_ref}/fmap/{subject_ref}_{file_name}.nii.gz -dlabel ../../{subject_ref}/fmap/{subject_ref}_{file_name}_label.nii.gz -param step=1,type=label,dof=Tx_Ty_Tz



--
ESTIMATE TRANSFORMATION FOR STEP #1

Apply transformation from previous step
/Users/evaalonsoortiz/sct_6.2/bin/isct_antsApplyTransforms -d 3 -i src_label.nii -o src_label_reg.nii -t warp_forward_0.nii.gz -r dest_label_RPI.nii -n NearestNeighbor # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-52-02_register-wrapper_oeuuqm9a
Registration parameters:
  type ........... label
  algo ........... syn
  slicewise ...... 0
  metric ......... MeanSquares
  samplStrategy .. None
  samplPercent ... 0.2
  iter ........... 10
  smooth ......... 0
  laplacian ...... 0
  shrink ......... 1
  gradStep ....... 0.5
  deformation .... 1x1x0
  init ........... 
  poly ........... 5
  filter_size .... 5
  dof ............ Tx_Ty_Tz
  smoothWarpXY ... 2
  rot_method ..... pca
Parameter 'algo=syn' has no effect for 'type=label' registration.
Labels src: [[-3.617250442504883, 13.477035522460938, 39.09071350097656]]
Labels dest: [[-11.117250442504883, 50.97703552246094, -3.40

Labels src: [[-3.9172534942626953, 6.9770507789802405, 37.590698240380505]]
Labels dest: [[-11.117252349853516, 45.97705077917857, -2.9093017594285016]]
Degrees of freedom (dof): Tx_Ty_Tz
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 50
Matrix:
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]
Center:
 [-11.11725235  45.97705078  -2.90930176]
Translation:
 [[  7.19999886 -39.          40.5       ]]

Concatenate transformations...
/Users/evaalonsoortiz/sct_6.2/bin/isct_ComposeMultiTransform 3 warp_src2dest.nii.gz -R dest.nii warp_forward_1.txt warp_forward_0.nii.gz # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-52-08_register-wrapper_k08noshc
/Users/evaalonsoortiz/sct_6.2/bin/isct_ComposeMultiTransform 3 warp_dest2src.nii.gz -R src.nii -i warp_forward_1.txt warp_inverse_0.nii.gz # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-52-08_registe

/Users/evaalonsoortiz/sct_6.2/bin/isct_ComposeMultiTransform 3 warp_dest2src.nii.gz -R src.nii -i warp_forward_1.txt warp_inverse_0.nii.gz # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-52-17_register-wrapper_pvca0be_

Apply transfo source --> dest...
/Users/evaalonsoortiz/sct_6.2/bin/isct_antsApplyTransforms -d 3 -i src.nii -o src_reg.nii -t warp_src2dest.nii.gz -r dest.nii -n Linear # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-52-17_register-wrapper_pvca0be_

Apply transfo dest --> source...
/Users/evaalonsoortiz/sct_6.2/bin/isct_antsApplyTransforms -d 3 -i dest.nii -o dest_reg.nii -t warp_dest2src.nii.gz -r src.nii -n Linear # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-52-17_register-wrapper_pvca0be_

Generate output files...
File sub-MPI_TFLTB1map_reg.nii.gz already exists. Deleting it..
File created: sub-MPI_TFLTB1map_reg.nii.gz
File warp_sub-MPI_TFLTB1map2sub-CRMBM_TFLTB1map.nii


Generate output files...
File sub-MPI_DREAMTB1avgB1map_reg.nii.gz already exists. Deleting it..
File created: sub-MPI_DREAMTB1avgB1map_reg.nii.gz
File warp_sub-MPI_DREAMTB1avgB1map2sub-CRMBM_DREAMTB1avgB1map.nii.gz already exists. Deleting it..
mv /var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-52-22_register-wrapper_5bihy9bz/warp_src2dest.nii.gz warp_sub-MPI_DREAMTB1avgB1map2sub-CRMBM_DREAMTB1avgB1map.nii.gz
File created: warp_sub-MPI_DREAMTB1avgB1map2sub-CRMBM_DREAMTB1avgB1map.nii.gz
File sub-CRMBM_DREAMTB1avgB1map_reg.nii.gz already exists. Deleting it..
File created: sub-CRMBM_DREAMTB1avgB1map_reg.nii.gz
File warp_sub-CRMBM_DREAMTB1avgB1map2sub-MPI_DREAMTB1avgB1map.nii.gz already exists. Deleting it..
mv /var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-52-22_register-wrapper_5bihy9bz/warp_dest2src.nii.gz warp_sub-CRMBM_DREAMTB1avgB1map2sub-MPI_DREAMTB1avgB1map.nii.gz
File created: warp_sub-CRMBM_DREAMTB1avgB1map2sub-MPI_DREAMTB1avgB1map.nii.gz


File created: sub-CRMBM_acq-coilQaSagLarge_SNR_T0000_reg.nii.gz
File warp_sub-CRMBM_acq-coilQaSagLarge_SNR_T00002sub-MPI_acq-coilQaSagLarge_SNR_T0000.nii.gz already exists. Deleting it..
mv /var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-52-27_register-wrapper_m5r3n714/warp_dest2src.nii.gz warp_sub-CRMBM_acq-coilQaSagLarge_SNR_T00002sub-MPI_acq-coilQaSagLarge_SNR_T0000.nii.gz
File created: warp_sub-CRMBM_acq-coilQaSagLarge_SNR_T00002sub-MPI_acq-coilQaSagLarge_SNR_T0000.nii.gz

Remove temporary files...
rm -rf /var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-52-27_register-wrapper_m5r3n714

Finished! Elapsed time: 5s

Done! To view results, type:
fsleyes sub-MPI_acq-coilQaSagLarge_SNR_T0000.nii.gz sub-CRMBM_acq-coilQaSagLarge_SNR_T0000_reg.nii.gz &


Done! To view results, type:
fsleyes ../../sub-CRMBM/fmap/sub-CRMBM_acq-coilQaSagLarge_SNR_T0000.nii.gz sub-MPI_acq-coilQaSagLarge_SNR_T0000_reg.nii.gz &

👉 PROCESSING: sub-MSSM

--
Spinal Cord Toolbox (

...


--
ESTIMATE TRANSFORMATION FOR STEP #1

Apply transformation from previous step
/Users/evaalonsoortiz/sct_6.2/bin/isct_antsApplyTransforms -d 3 -i src_label.nii -o src_label_reg.nii -t warp_forward_0.nii.gz -r dest_label_RPI.nii -n NearestNeighbor # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-53-00_register-wrapper_nf0mfi_1
Registration parameters:
  type ........... label
  algo ........... syn
  slicewise ...... 0
  metric ......... MeanSquares
  samplStrategy .. None
  samplPercent ... 0.2
  iter ........... 10
  smooth ......... 0
  laplacian ...... 0
  shrink ......... 1
  gradStep ....... 0.5
  deformation .... 1x1x0
  init ........... 
  poly ........... 5
  filter_size .... 5
  dof ............ Tx_Ty_Tz
  smoothWarpXY ... 2
  rot_method ..... pca
Parameter 'algo=syn' has no effect for 'type=label' registration.
Labels src: [[1.3827495574951172, 5.9770355224609375, 29.090713500976562]]
Labels dest: [[-11.117250442504883, 50.97703552246094, -3.4

Registration parameters:
  type ........... label
  algo ........... syn
  slicewise ...... 0
  metric ......... MeanSquares
  samplStrategy .. None
  samplPercent ... 0.2
  iter ........... 10
  smooth ......... 0
  laplacian ...... 0
  shrink ......... 1
  gradStep ....... 0.5
  deformation .... 1x1x0
  init ........... 
  poly ........... 5
  filter_size .... 5
  dof ............ Tx_Ty_Tz
  smoothWarpXY ... 2
  rot_method ..... pca
Parameter 'algo=syn' has no effect for 'type=label' registration.
Labels src: [[3.282745361328125, 0.9770507790316589, 27.090698240351117]]
Labels dest: [[-11.117252349853516, 45.97705077917857, -2.9093017594285016]]
Degrees of freedom (dof): Tx_Ty_Tz
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 50
Matrix:
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]
Center:
 [-11.11725235  45.97705078  -2.90930176]
Translation:
 [[ 14.39999771 -45.          30.        ]]

Concatena

Labels src: [[1.2492046356201172, 50.143698930740356, 25.062935829162598]]
Labels dest: [[-11.087461948394775, 50.143698930740356, -3.270395278930664]]
Degrees of freedom (dof): Tx_Ty_Tz
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 3
         Function evaluations: 107
Matrix:
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]
Center:
 [-11.08746195  50.14369893  -3.27039528]
Translation:
 [[1.23366666e+01 1.09411934e-16 2.83333311e+01]]

Concatenate transformations...
/Users/evaalonsoortiz/sct_6.2/bin/isct_ComposeMultiTransform 3 warp_src2dest.nii.gz -R dest.nii warp_forward_1.txt warp_forward_0.nii.gz # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-53-14_register-wrapper_zaprbvw4
/Users/evaalonsoortiz/sct_6.2/bin/isct_ComposeMultiTransform 3 warp_dest2src.nii.gz -R src.nii -i warp_forward_1.txt warp_inverse_0.nii.gz # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-53-14_r

/Users/evaalonsoortiz/sct_6.2/bin/isct_ComposeMultiTransform 3 warp_dest2src.nii.gz -R src.nii -i warp_forward_1.txt warp_inverse_0.nii.gz # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-53-19_register-wrapper_ib251uuf

Apply transfo source --> dest...
/Users/evaalonsoortiz/sct_6.2/bin/isct_antsApplyTransforms -d 3 -i src.nii -o src_reg.nii -t warp_src2dest.nii.gz -r dest.nii -n Linear # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-53-19_register-wrapper_ib251uuf

Apply transfo dest --> source...
/Users/evaalonsoortiz/sct_6.2/bin/isct_antsApplyTransforms -d 3 -i dest.nii -o dest_reg.nii -t warp_dest2src.nii.gz -r src.nii -n Linear # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-53-19_register-wrapper_ib251uuf

Generate output files...
File sub-UCL_DREAMTB1avgB1map_reg.nii.gz already exists. Deleting it..
File created: sub-UCL_DREAMTB1avgB1map_reg.nii.gz
File warp_sub-UCL_DREAMTB1avgB1map2su


Apply transfo dest --> source...
/Users/evaalonsoortiz/sct_6.2/bin/isct_antsApplyTransforms -d 3 -i dest.nii -o dest_reg.nii -t warp_dest2src.nii.gz -r src.nii -n Linear # in /private/var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-53-25_register-wrapper_ylylssm3

Generate output files...
File sub-UCL_acq-coilQaSagLarge_SNR_T0000_reg.nii.gz already exists. Deleting it..
File created: sub-UCL_acq-coilQaSagLarge_SNR_T0000_reg.nii.gz
File warp_sub-UCL_acq-coilQaSagLarge_SNR_T00002sub-CRMBM_acq-coilQaSagLarge_SNR_T0000.nii.gz already exists. Deleting it..
mv /var/folders/s5/9kk838_10bl825z692jjscs80000gn/T/sct_2024-06-25_14-53-25_register-wrapper_ylylssm3/warp_src2dest.nii.gz warp_sub-UCL_acq-coilQaSagLarge_SNR_T00002sub-CRMBM_acq-coilQaSagLarge_SNR_T0000.nii.gz
File created: warp_sub-UCL_acq-coilQaSagLarge_SNR_T00002sub-CRMBM_acq-coilQaSagLarge_SNR_T0000.nii.gz
File sub-CRMBM_acq-coilQaSagLarge_SNR_T0000_reg.nii.gz already exists. Deleting it..
File created: sub-CRMBM_a

In [ ]:
# Generate syntax to open registered data in FSLeyes. This should be run from within the `data-phantom/` folder.
for file_name in file_names:
    print(f"\n👉 CHECKING REGISTRATION FOR: {file_name}\n")
    cmd = f"fsleyes sub-CRMBM/fmap/sub-CRMBM_{file_name}.nii.gz"
    for subject in [other_subjects for other_subjects in subjects if other_subjects != subject_ref]:
        file_reg = os.path.join(path_data, subject, "fmap", f"{subject}_{file_name}_reg.nii.gz")
        cmd = f"{cmd} {file_reg}"
    print(cmd+" &")

## Extract values in mask and generate figures 

In [ ]:
# Extract B1+ and SNR along the spinal cord mask and save data to CSV files

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    
    fname_result_TFLb1plus = os.path.join(path_results, f"{subject}_TFLTB1map.csv")
    fname_result_DREAMb1plus = os.path.join(path_results, f"{subject}_DREAMTB1avgB1map.csv")
    fname_result_SNR = os.path.join(path_results, f"{subject}_acq-coilQaSagLarge_SNR_T0000.csv")

    # For sites other than sub-CRMBM, use the registered qMRI map for metrics extraction
    if subject=='sub-CRMBM':
        file_suffix = ''
    else:
        file_suffix = '_reg'
        
    # Extract metrics within mask defined in the sub-CRMBM space
    !sct_extract_metric -i {subject}_TFLTB1map{file_suffix}.nii.gz -f ../../sub-CRMBM/fmap/mask_spinalcord_TFL.nii.gz -method wa -z 19:127 -perslice 1 -o "{fname_result_TFLb1plus}"     
    !sct_extract_metric -i {subject}_DREAMTB1avgB1map{file_suffix}.nii.gz -f ../../sub-CRMBM/fmap/mask_spinalcord_DREAM.nii.gz -method wa -z 14:62 -perslice 1 -o "{fname_result_DREAMb1plus}"     
    !sct_extract_metric -i {subject}_acq-coilQaSagLarge_SNR_T0000{file_suffix}.nii.gz -f ../../sub-CRMBM/fmap/mask_spinalcord_SNR.nii.gz -method wa -z 151:368 -perslice 1 -o "{fname_result_SNR}"     

In [ ]:
# Loop across metrics and generate figure

# Define figure legend/labels assuming the following order: B1+ TFL, B1+ DREAM, SNR
fig_types = ["TFL B1+ along the S/I axis", "DREAM B1+ along the S/I axis","SNR along the S/I axis"]
file_names = ["TFLTB1map", "DREAMTB1avgB1map","acq-coilQaSagLarge_SNR_T0000"]
ylabels = ["B1+ [nT/V]", "B1+ [nT/V]", "a.u."]

for fig_type, file_name, ylabel in zip(fig_types, file_names, ylabels):
    fig=plt.plot()
    for subject in subjects:
        os.chdir(os.path.join(path_data, f"{subject}", "fmap"))
        # Read metric data
        file_csv = os.path.join(path_results, f"{subject}_{file_name}.csv")
        df = pd.read_csv(file_csv)
        data = df['WA()']
        # Create figure
        plt.plot(data,label=subject)

    plt.legend()
    plt.ylabel(ylabel)
    plt.title(fig_type)
    plt.show()

In [ ]:
###########################################################################################################
# generate figures for B1+, SNR, and 1/g-factor maps obtained at each site 
###########################################################################################################

In [ ]:
# Compute the average and maximum 1/g factor in a central ROI
# Display 1/g factor maps and T2w (coil-combined) maps for each site (MSSM excluded, see: https://github.com/spinal-cord-7t/coil-qc-code/issues/61)

In [ ]:
sites = ["CRMBM", "MGH", "MNI", "MPI", "NTNU", "UCL"] # MSSM excluded, see: https://github.com/spinal-cord-7t/coil-qc-code/issues/61

# map types
map_types = ["acq-coilQaSagSmall_GFactor", "T2starw"]
# legend types
legend_types = ["[1/g]", "[arb]"]

mean_gfac = {}
max_gfac = {}

for map_type, legend_type in zip(map_types,legend_types):
    
    # Create a figure with multiple subplots
    fig, axes = plt.subplots(2, 3, figsize=(10, 8))
    font_size = 12
    axes=axes.flatten() 
        
    for i,site in enumerate(sites):
        # Load data
        if map_type=="T2starw":
            os.chdir(os.path.join(path_data, f"sub-{site}", "anat"))
            map=nib.load(f"sub-{site}_{map_type}.nii.gz")
            data=map.get_fdata()[:,:,round(map.get_fdata().shape[2]/2)]
        else:         
            os.chdir(os.path.join(path_data, f"sub-{site}", "fmap"))
            map=nib.load(f"sub-{site}_{map_type}.nii.gz")
            data=(map.get_fdata()[64:191,64:191,round(map.get_fdata().shape[2]/2),5])/1000
            
            gfac_data=(map.get_fdata()[round(map.get_fdata().shape[0]/2)-10:round(map.get_fdata().shape[0]/2)+10,round(map.get_fdata().shape[1]/2)-10:round(map.get_fdata().shape[1]/2)+10,round(map.get_fdata().shape[2]/2),5])
            mean_gfac[site]=np.nanmean(gfac_data)/1000
            max_gfac[site]=np.max(gfac_data)/1000
        
        # Plot    
        splot=axes[i]
        dynmin = 0
        if map_type=="acq-coilQaSagSmall_GFactor":
            dynmax = 1
            axes[-1].axis('off')
            splot.text(0, 3, r'mean gfac='+str(round(mean_gfac[site],2)), size=10)
            splot.text(0, 12, r'max gfac='+str(round(max_gfac[site],2)), size=10)
            
            x = [data.shape[0]/2-10, data.shape[1]/2-10] 
            y = [data.shape[0]/2-10, data.shape[1]/2+10] 
            splot.plot(x, y, color="black", linewidth=2) 
            
            x = [data.shape[0]/2-10, data.shape[1]/2+10] 
            y = [data.shape[0]/2+10, data.shape[1]/2+10] 
            splot.plot(x, y, color="black", linewidth=2) 

            x = [data.shape[0]/2+10, data.shape[1]/2+10] 
            y = [data.shape[0]/2+10, data.shape[1]/2-10] 
            splot.plot(x, y, color="black", linewidth=2)

            x = [data.shape[0]/2+10, data.shape[1]/2-10] 
            y = [data.shape[0]/2-10, data.shape[1]/2-10] 
            splot.plot(x, y, color="black", linewidth=2) 
            
        elif map_type=="T2starw":
            dynmax = 3000
            axes[-1].axis('off')
        
        im = splot.imshow((data.T), cmap='viridis', origin='lower',vmin=dynmin,vmax=dynmax) 
        splot.set_title(site, size=font_size)
        splot.axis('off')

    plt.tight_layout()

    # Colorbar
    # Assume that the colorbar should start at the bottom of the lower row of subplots and
    # extend to the top of the upper row of subplots
    cbar_bottom = 0.2  # This might need adjustment
    cbar_height = 0.6  # This represents the total height of both rows of subplots
    cbar_dist = 1.01
    cbar_ax = fig.add_axes([cbar_dist, cbar_bottom, 0.03, cbar_height])
    cbar = plt.colorbar(im, cax=cbar_ax)

    cbar_ax.set_title(legend_type, size=12)
    plt.show()


In [ ]:
# Display TFL and DREAM B1+ maps, and SNR maps for each site

In [ ]:
sites = ["CRMBM", "MGH", "MNI", "MPI", "MSSM", "NTNU", "UCL"]

# map types
map_types = ["TFLTB1map", "DREAMTB1avgB1map", "acq-coilQaSagLarge_SNR_T0000"]

# legend types
legend_types = ["[nT/V]", "[nT/V]", "[arb]"]


for map_type, legend_type in zip(map_types,legend_types):
    
    # Create a figure with multiple subplots
    fig, axes = plt.subplots(2, 4, figsize=(10, 8))
    font_size = 12
    axes=axes.flatten() 
        
    for i,site in enumerate(sites):
        # Load data
        os.chdir(os.path.join(path_data, f"sub-{site}", "fmap"))
        if site=='CRMBM':
            map=nib.load(f"sub-{site}_{map_type}.nii.gz")
            if map_type=="TFLTB1map":
                mask=nib.load(f"mask_spinalcord_TFL.nii.gz")
            elif map_type=="DREAMTB1avgB1map":
                mask=nib.load(f"mask_spinalcord_DREAM.nii.gz")
            else: 
                mask=nib.load(f"mask_spinalcord_SNR.nii.gz")
        else:
            map=nib.load(f"sub-{site}_{map_type}_reg.nii.gz")
            if map_type=="TFLTB1map":
                mask=nib.load(f"../../sub-CRMBM/fmap/mask_spinalcord_TFL.nii.gz")
            elif map_type=="DREAMTB1avgB1map":
                mask=nib.load(f"../../sub-CRMBM/fmap/mask_spinalcord_DREAM.nii.gz")
            else: 
                mask=nib.load(f"../../sub-CRMBM/fmap/mask_spinalcord_SNR.nii.gz")

        
        if map_type=="acq-coilQaSagLarge_SNR_T0000":
            data=map.get_fdata()[115:350,151:368,round(map.get_fdata().shape[2]/2)]
            mask_data=mask.get_fdata()[115:350,151:368,round(mask.get_fdata().shape[2]/2)]
        elif map_type=="TFLTB1map":
            data=map.get_fdata()[19:83,19:127,round(map.get_fdata().shape[2]/2)]
            mask_data=mask.get_fdata()[19:83,19:127,22]
        elif map_type=="DREAMTB1avgB1map":
            data=map.get_fdata()[18:79,14:62,round(map.get_fdata().shape[2]/2)]
            mask_data=mask.get_fdata()[18:79,14:62,round(mask.get_fdata().shape[2]/2)]
    
        # Plot
        dynmin = 0 
        if map_type=="acq-coilQaSagLarge_SNR_T0000":
            dynmax = 300
        else:
            dynmax = 70    
            
        splot=axes[i]
        im1 = splot.imshow((data.T), cmap='viridis', origin='lower',vmin=dynmin,vmax=dynmax)
        im2 = splot.imshow((mask_data.T), 'gray', origin='lower', alpha=0.2)
        splot.set_title(site, size=font_size)
        splot.axis('off')

    for site in sites:
        
        os.chdir(os.path.join(path_data, f"sub-{site}", "fmap"))
            
        file_csv = os.path.join(path_results, f"sub-{site}_{map_type}.csv")
        df = pd.read_csv(file_csv)
        data = df['WA()']
        axes[-1].plot(data,label=site)
         
    axes[-1].legend(fontsize=10,loc='center left', bbox_to_anchor=(1.3, 0.5))
    axes[-1].yaxis.set_label_position("right")
    axes[-1].yaxis.tick_right()
        
    if map_type=="acq-coilQaSagLarge_SNR_T0000":
        axes[-1].set_ylabel("[arb]")
    else:
        axes[-1].set_ylabel("B1+ [nT/V]")

    
    plt.tight_layout()
    #plt.subplots_adjust(wspace=0.2, hspace=0.1)

    # Colorbar
    # Assume that the colorbar should start at the bottom of the lower row of subplots and
    # extend to the top of the upper row of subplots
    cbar_bottom = 0.2  # This might need adjustment
    cbar_height = 0.6  # This represents the total height of both rows of subplots
    if (map_type == "TFLTB1map" or map_type == "DREAMTB1avgB1map" or map_type == "acq-coilQaSagLarge_SNR_T0000"):
        cbar_dist = -0.1
    else:
        cbar_dist = 1.01
    cbar_ax = fig.add_axes([cbar_dist, cbar_bottom, 0.03, cbar_height])
    cbar = plt.colorbar(im1, cax=cbar_ax)

    cbar_ax.set_title(legend_type, size=12)
    plt.show()


In [ ]:
###########################################################################################################
# generate tiled figure with individual channel GRE maps 
###########################################################################################################

In [ ]:
sites = ["CRMBM", "MGH", "MNI", "MPI", "MSSM", "NTNU", "UCL"]
        
for i,site in enumerate(sites):

    gre_files=sorted(glob.glob(os.path.join(path_data, f"sub-{site}", "anat", '*uncombined*.nii.gz')))
        
    #Tiled figure in a five-row layout
    rows=int(np.ceil(len(gre_files)/4))
    cols=int(np.ceil(len(gre_files)/rows))

    fig=plt.figure(figsize=(15, 20))
    
    ax = fig.subplots(rows,cols,squeeze=True)
    
    for row in range(rows):
        for col in range(cols):

            i = row*cols+col

            if i < len(gre_files):
            
                #read in files
                data_to_plot=(nib.load(gre_files[i])).get_fdata() #load in nifti object, get only image data
                data_to_plot=np.rot90(data_to_plot[:,:,int(np.floor(data_to_plot.shape[2]/2))]) #central slice
           
                ax[row,col].imshow(data_to_plot,cmap=plt.cm.gray,clim=[0, 300])
                ax[row,col].text(0.5, 0.05, 'Rx channel : ' + str(i+1),horizontalalignment='center', transform=ax[row,col].transAxes,color='white',fontsize=17)
                ax[row,col].axis('off')

    plt.axis('off')
    plt.subplots_adjust(hspace=0,wspace=0)
    fig.suptitle(site, fontsize=20, y=0.9)


## Finished

In [ ]:
# Indicate duration of data processing

end_time = datetime.now()
total_time = (end_time - start_time).total_seconds()

# Convert seconds to a timedelta object
total_time_delta = timedelta(seconds=total_time)

# Format the timedelta object to a string
formatted_time = str(total_time_delta)

# Pad the string representation if less than an hour
formatted_time = formatted_time.rjust(8, '0')

print(f"Total Runtime [hour:min:sec]: {formatted_time}")